In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
from datetime import timedelta
import numpy as np

pd.set_option('display.max_columns', None)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
colors = sns.color_palette('pastel')

In [2]:
df_click = pd.read_csv('D:/Section5/TeamProject2/model_new_session_id.csv')
df_trans_new = pd.read_csv('D:/Section5/TeamProject2/csv/transaction_new.csv')
df_customer = pd.read_csv('D:/Section5/TeamProject2/csv/customer.csv')

In [3]:
df_trans_user = df_trans_new.merge(df_customer, on = 'customer_id', how = 'left')

In [4]:
df_click = df_click.rename(columns={'session_id': 'new_session_id'})
df_click['session_id'] = df_click['new_session_id'].apply(lambda x: x[:36])

In [5]:
# session_id 열을 기준으로 df와 df_trans_new를 병합
merged_df = pd.merge(df_click, df_trans_user, on='session_id', how='left')
merged_df['event_time'] = pd.to_datetime(merged_df['event_time'])
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21130155 entries, 0 to 21130154
Data columns (total 43 columns):
 #   Column                  Dtype              
---  ------                  -----              
 0   new_session_id          object             
 1   event_name              object             
 2   event_time              datetime64[ns, UTC]
 3   event_id                object             
 4   traffic_source          object             
 5   product_id_x            float64            
 6   quantity_x              float64            
 7   item_price_x            float64            
 8   payment_status_x        object             
 9   search_keywords         object             
 10  promo_code_x            object             
 11  promo_amount_x          float64            
 12  duration_time           object             
 13  session_id              object             
 14  created_at              object             
 15  customer_id             float64            
 16

In [6]:
merged_df['first_join_date'] = pd.to_datetime(merged_df['first_join_date'])

In [7]:
merged_df = merged_df[['new_session_id', 'event_name', 'product_id_x', 'event_time', 'total_amount', 'traffic_source', 'payment_method', 'shipment_date_limit', 'payment_status_x','shipment_fee', 'search_keywords', 'promo_code_x', 'promo_amount_x', 'customer_id', 'first_join_date']]

In [8]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21130155 entries, 0 to 21130154
Data columns (total 15 columns):
 #   Column               Dtype              
---  ------               -----              
 0   new_session_id       object             
 1   event_name           object             
 2   product_id_x         float64            
 3   event_time           datetime64[ns, UTC]
 4   total_amount         float64            
 5   traffic_source       object             
 6   payment_method       object             
 7   shipment_date_limit  object             
 8   payment_status_x     object             
 9   shipment_fee         float64            
 10  search_keywords      object             
 11  promo_code_x         object             
 12  promo_amount_x       float64            
 13  customer_id          float64            
 14  first_join_date      datetime64[ns]     
dtypes: datetime64[ns, UTC](1), datetime64[ns](1), float64(5), object(8)
memory usage: 2.4+ GB


In [9]:
time_df = merged_df[['customer_id', 'new_session_id', 'event_time', 'first_join_date', 'traffic_source']].drop_duplicates(subset='new_session_id', keep = 'first')

In [10]:
time_df['created_at'] = time_df['event_time'].dt.strftime('%Y-%m-%d')
time_df.head(5)

,customer_id,new_session_id,event_time,first_join_date,traffic_source,created_at
0,74433.0,00001f29-3ede-4445-95f0-0178934e32ba_1,2019-05-17 11:51:52.276396+00:00,2017-02-05,MOBILE,2019-05-17
1,74433.0,00001f29-3ede-4445-95f0-0178934e32ba_2,2019-05-17 19:54:53.276396+00:00,2017-02-05,MOBILE,2019-05-17
3,74433.0,00001f29-3ede-4445-95f0-0178934e32ba_3,2019-05-18 03:58:25.276396+00:00,2017-02-05,MOBILE,2019-05-18
4,74433.0,00001f29-3ede-4445-95f0-0178934e32ba_4,2019-05-18 12:02:44.276396+00:00,2017-02-05,MOBILE,2019-05-18
5,74433.0,00001f29-3ede-4445-95f0-0178934e32ba_5,2019-05-18 20:04:54.276396+00:00,2017-02-05,MOBILE,2019-05-18


In [11]:
time_df = time_df.groupby(['customer_id', 'created_at'])[['first_join_date','new_session_id']].first().reset_index()

In [12]:
time_df['created_at'] = pd.to_datetime(time_df['created_at'])

In [13]:
time_df['diff'] = time_df.groupby('customer_id')['created_at'].diff().shift(-1)

In [14]:
time_df = time_df[time_df['diff'].isna()]

In [15]:
time_df

,customer_id,created_at,first_join_date,new_session_id,diff
321,3.0,2022-06-26,2018-08-25,a857dcbe-5ef9-47a7-9fd6-478de1ffb635_5563287,NaT
409,8.0,2022-05-15,2021-02-13,1b8848eb-5ebc-4cbf-bf42-64c86b62c9d6_907932,NaT
447,9.0,2022-05-28,2021-06-26,6b23a139-8b85-4e68-a270-082ddf127cc4_3529286,NaT
448,11.0,2022-03-07,2022-03-06,5f20fb20-8e35-4e30-a9d9-fc6a07d7835c_3128683,NaT
487,15.0,2022-01-05,2019-09-28,cb1bb055-7431-4a77-b3e5-c896a9462aeb_6712949,NaT
...,...,...,...,...,...
4395744,99992.0,2022-05-25,2020-06-30,0ca99ab4-4237-4d6b-b858-7b6aa4b3b44a_417016,NaT
4395984,99995.0,2022-07-29,2020-08-16,be458630-6c39-4cf7-931d-010194e350d0_6288275,NaT
4395985,99997.0,2020-04-25,2020-04-25,17e0c216-a441-4aff-986d-875fda0c1d8c_784765,NaT
4396087,99998.0,2022-01-25,2018-07-01,4fc4d81b-4b29-4aef-bd1f-263c373950d1_2625129,NaT


In [16]:
time_df['customer_id'].nunique()

50704

In [17]:
time_df['diff'].describe()

count      0
mean     NaT
std      NaT
min      NaT
25%      NaT
50%      NaT
75%      NaT
max      NaT
Name: diff, dtype: object

In [18]:
# check_df에서 diff칼럼이 15일 이상인 행 추출
mask = time_df['diff'] > pd.Timedelta(days=5)
# target 열 생성
time_df['target'] = np.where(mask, 1, 0)

In [19]:
time_df['target'].value_counts(normalize=True)

target
0    1.0
Name: proportion, dtype: float64

In [20]:
(time_df.groupby('customer_id')['target'].sum()/time_df.groupby('customer_id')['target'].size() * 100).describe()

count    50704.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: target, dtype: float64

In [21]:
# time_df.to_csv('target.csv')

In [22]:
merged_df.head(5)

,new_session_id,event_name,product_id_x,event_time,total_amount,traffic_source,payment_method,shipment_date_limit,payment_status_x,shipment_fee,search_keywords,promo_code_x,promo_amount_x,customer_id,first_join_date
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,NaN,2019-05-17 11:51:52.276396+00:00,1427802.0,MOBILE,Credit Card,2019-05-21T13:04:58.898492Z,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,33855.0,2019-05-17 19:54:53.276396+00:00,1427802.0,MOBILE,Credit Card,2019-05-21T13:04:58.898492Z,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,NaN,2019-05-17 19:55:12.276396+00:00,1427802.0,MOBILE,Credit Card,2019-05-21T13:04:58.898492Z,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,NaN,2019-05-18 03:58:25.276396+00:00,1427802.0,MOBILE,Credit Card,2019-05-21T13:04:58.898492Z,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,NaN,2019-05-18 12:02:44.276396+00:00,1427802.0,MOBILE,Credit Card,2019-05-21T13:04:58.898492Z,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05


In [23]:
# ADD_TO_CART COUNT
merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'ADD_TO_CART'].groupby('new_session_id')['event_time'].count(), on = 'new_session_id', how = 'left')
merged_df['event_time_y'] = merged_df['event_time_y'].fillna(0)
merged_df = merged_df.rename(columns={'event_time_y': 'ADD_TO_CART_COUNT'})

In [24]:
merged_df = merged_df.rename(columns={'event_time_x': 'event_time'})

In [25]:
# SEARCH COUNT
merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'SEARCH'].groupby('new_session_id')['event_time'].count(), on = 'new_session_id', how = 'left')
merged_df['event_time_y'] = merged_df['event_time_y'].fillna(0)
merged_df = merged_df.rename(columns={'event_time_y': 'SEARCH_COUNT'})
merged_df = merged_df.rename(columns={'event_time_x': 'event_time'})

In [26]:
# ADD_PROMO COUNT
merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'ADD_PROMO'].groupby('new_session_id')['event_time'].count(), on = 'new_session_id', how = 'left')
merged_df['event_time_y'] = merged_df['event_time_y'].fillna(0)
merged_df = merged_df.rename(columns={'event_time_y': 'ADD_PROMO_COUNT'})
merged_df = merged_df.rename(columns={'event_time_x': 'event_time'})

In [27]:
# BOOKING COUNT
merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'BOOKING'].groupby('new_session_id')['event_time'].count(), on = 'new_session_id', how = 'left')
merged_df['event_time_y'] = merged_df['event_time_y'].fillna(0)
merged_df = merged_df.rename(columns={'event_time_y': 'BOOKING_COUNT'})
merged_df = merged_df.rename(columns={'event_time_x': 'event_time'})

In [28]:
# ITEM_DETAIL COUNT
merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'ITEM_DETAIL'].groupby('new_session_id')['event_time'].count(), on = 'new_session_id', how = 'left')
merged_df['event_time_y'] = merged_df['event_time_y'].fillna(0)
merged_df = merged_df.rename(columns={'event_time_y': 'ITEM_DETAIL_COUNT'})
merged_df = merged_df.rename(columns={'event_time_x': 'event_time'})

In [29]:
# merged_df = merged_df.merge(merged_df[merged_df['event_name'] == 'SEARCH'].groupby('new_session_id')['event_time_x'].count(), on = 'new_session_id', how = 'left')
# merged_df.head(5)

In [30]:
# merged_df.to_csv('check_num_events.csv')

In [31]:
time_df.head(5)

,customer_id,created_at,first_join_date,new_session_id,diff,target
321,3.0,2022-06-26,2018-08-25,a857dcbe-5ef9-47a7-9fd6-478de1ffb635_5563287,NaT,0
409,8.0,2022-05-15,2021-02-13,1b8848eb-5ebc-4cbf-bf42-64c86b62c9d6_907932,NaT,0
447,9.0,2022-05-28,2021-06-26,6b23a139-8b85-4e68-a270-082ddf127cc4_3529286,NaT,0
448,11.0,2022-03-07,2022-03-06,5f20fb20-8e35-4e30-a9d9-fc6a07d7835c_3128683,NaT,0
487,15.0,2022-01-05,2019-09-28,cb1bb055-7431-4a77-b3e5-c896a9462aeb_6712949,NaT,0


In [32]:
# df_share = time_df.merge(df_click, on = 'new_session_id', how = 'left')

In [33]:
# df_share.to_csv('merge_to_share.csv')

In [34]:
# df_share.head(5)

In [35]:
merged_df['shipment_date_limit'] = pd.to_datetime(merged_df['shipment_date_limit'])

In [36]:
merged_df['limit_ship'] = merged_df[merged_df['event_name'] == 'BOOKING']['shipment_date_limit'] - merged_df[merged_df['event_name'] == 'BOOKING']['event_time']

In [37]:
merged_df = merged_df.drop(['shipment_date_limit'], axis = 1)

In [38]:
merged_df.head(10)

,new_session_id,event_name,product_id_x,event_time,total_amount,traffic_source,payment_method,payment_status_x,shipment_fee,search_keywords,promo_code_x,promo_amount_x,customer_id,first_join_date,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,NaN,2019-05-17 11:51:52.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,0.0,0.0,NaT
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,33855.0,2019-05-17 19:54:53.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,1.0,0.0,0.0,0.0,0.0,NaT
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,NaN,2019-05-17 19:55:12.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,1.0,0.0,0.0,0.0,0.0,NaT
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,NaN,2019-05-18 03:58:25.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,0.0,0.0,NaT
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,NaN,2019-05-18 12:02:44.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,0.0,0.0,NaT
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,NaN,2019-05-18 20:04:54.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,0.0,1.0,NaT
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,NaN,2019-05-19 04:07:20.276396+00:00,1427802.0,MOBILE,Credit Card,NaN,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,0.0,0.0,NaT
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,NaN,2019-05-19 20:17:04.276396+00:00,1427802.0,MOBILE,Credit Card,Success,0.0,NaN,NaN,NaN,74433.0,2017-02-05,0.0,0.0,0.0,1.0,0.0,1 days 16:47:54.622096
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,NaN,2019-08-24 05:47:16.849738+00:00,339790.0,MOBILE,LinkAja,NaN,0.0,NaN,NaN,NaN,43805.0,2019-08-16,1.0,0.0,0.0,1.0,0.0,NaT
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,NaN,2019-08-24 05:59:38.849738+00:00,339790.0,MOBILE,LinkAja,NaN,0.0,NaN,NaN,NaN,43805.0,2019-08-16,1.0,0.0,0.0,1.0,0.0,NaT


In [39]:
merged_df['session_id'] = merged_df['new_session_id'].apply(lambda x: x[:36])

In [40]:
merged_df.loc[merged_df['event_name'] != 'BOOKING', 'total_amount'] = 0

In [41]:
merged_df[merged_df['total_amount']!= 0]['total_amount'].describe()

count    1.254578e+06
mean     1.181572e+06
std      1.728944e+06
min      1.089800e+04
25%      2.466450e+05
50%      4.558770e+05
75%      1.360496e+06
max      2.350449e+07
Name: total_amount, dtype: float64

In [42]:
max_amounts = merged_df.groupby('new_session_id')['total_amount'].max().reset_index()
merged_df = merged_df.merge(max_amounts, on='new_session_id', suffixes=('', '_max'))
merged_df.loc[merged_df['total_amount'] != merged_df['total_amount_max'], 'total_amount'] = merged_df['total_amount_max']
merged_df = merged_df.drop('total_amount_max', axis=1)

In [43]:
merged_df = merged_df.drop(['event_time', 'product_id_x', 'payment_status_x', 'search_keywords', 'first_join_date', 'promo_code_x'], axis=1)

In [44]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,customer_id,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,session_id
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,1.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,1.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,0.0,1.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,0.0,0.0,NaT,00001f29-3ede-4445-95f0-0178934e32ba
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,74433.0,0.0,0.0,0.0,1.0,0.0,1 days 16:47:54.622096,00001f29-3ede-4445-95f0-0178934e32ba
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,43805.0,1.0,0.0,0.0,1.0,0.0,NaT,00003eca-954b-4150-aee1-63fc62f395cf
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,43805.0,1.0,0.0,0.0,1.0,0.0,NaT,00003eca-954b-4150-aee1-63fc62f395cf


In [45]:
max_amounts = merged_df.groupby('new_session_id')['limit_ship'].max().reset_index()
merged_df = merged_df.merge(max_amounts, on='new_session_id', suffixes=('', '_max'))
merged_df.loc[merged_df['limit_ship'] != merged_df['limit_ship_max'], 'limit_ship'] = merged_df['limit_ship_max']
merged_df = merged_df.drop('limit_ship_max', axis=1)

In [46]:
merged_df = merged_df.drop(['customer_id','session_id'], axis=1)

In [47]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaT
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaT
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,1.0,NaT
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,1.0,0.0,1 days 16:47:54.622096
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,5 days 11:05:51.122116
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,5 days 11:05:51.122116


In [48]:
merged_df['PAGE_VIEW'] = merged_df.groupby('new_session_id')['event_name'].count()

In [49]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT,NaN
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaT,NaN
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,NaT,NaN
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT,NaN
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT,NaN
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,1.0,NaT,NaN
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaT,NaN
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,1.0,0.0,1 days 16:47:54.622096,NaN
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,5 days 11:05:51.122116,NaN
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,5 days 11:05:51.122116,NaN


In [50]:
def func(row):
    if row <= timedelta(days = 2):
        return 1
    elif row <= timedelta(days = 3):
        return 2
    elif row <= timedelta(days = 4):
        return 3
    elif row <= timedelta(days = 10):
        return 4
    else:
        return False
merged_df['limit_ship'] = merged_df['limit_ship'].apply(func)

In [51]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,NaN
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,False,NaN
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,False,NaN
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,NaN
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,NaN
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,1.0,False,NaN
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,NaN
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,1.0,0.0,1,NaN
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,NaN
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,NaN


In [52]:
merged_df['PAGE_VIEW'] = merged_df.groupby('new_session_id')['event_name'].transform('count')

In [53]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,1
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,False,2
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,False,2
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,1
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,1
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,1.0,False,1
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,False,1
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,1.0,0.0,1,1
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,4
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,4


In [54]:
merged_df.loc[merged_df['limit_ship'] == False, 'limit_ship'] = 0

In [55]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0,2
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0.0,NaN,1.0,0.0,0.0,0.0,0.0,0,2
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,1.0,0,1
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0.0,NaN,0.0,0.0,0.0,1.0,0.0,1,1
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,4
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0.0,NaN,1.0,0.0,0.0,1.0,0.0,4,4


In [56]:
merged_df.loc[merged_df['event_name'] != 'BOOKING', 'shipment_fee'] = 0

In [57]:
merged_df[merged_df['event_name'] == 'BOOKING']['shipment_fee'].describe()

count    1.254578e+06
mean     9.190831e+03
std      9.372016e+03
min      0.000000e+00
25%      0.000000e+00
50%      1.000000e+04
75%      1.000000e+04
max      5.000000e+04
Name: shipment_fee, dtype: float64

In [58]:
def func(x):
    if x <= 0:
        return 0 
    elif x <= 5000:
        return 1
    elif x <= 10000:
        return 2
    elif x <= 15000:
        return 3
    else:
        return 4


merged_df['shipment_fee'] = merged_df['shipment_fee'].apply(func)

In [59]:
merged_df.head(20)

,new_session_id,event_name,total_amount,traffic_source,payment_method,shipment_fee,promo_amount_x,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW
0,00001f29-3ede-4445-95f0-0178934e32ba_1,HOMEPAGE,0.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
1,00001f29-3ede-4445-95f0-0178934e32ba_2,ADD_TO_CART,0.0,MOBILE,Credit Card,0,NaN,1.0,0.0,0.0,0.0,0.0,0,2
2,00001f29-3ede-4445-95f0-0178934e32ba_2,HOMEPAGE,0.0,MOBILE,Credit Card,0,NaN,1.0,0.0,0.0,0.0,0.0,0,2
3,00001f29-3ede-4445-95f0-0178934e32ba_3,CLICK,0.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
4,00001f29-3ede-4445-95f0-0178934e32ba_4,CLICK,0.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
5,00001f29-3ede-4445-95f0-0178934e32ba_5,ITEM_DETAIL,0.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,0.0,1.0,0,1
6,00001f29-3ede-4445-95f0-0178934e32ba_6,SCROLL,0.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,0.0,0.0,0,1
7,00001f29-3ede-4445-95f0-0178934e32ba_7,BOOKING,1427802.0,MOBILE,Credit Card,0,NaN,0.0,0.0,0.0,1.0,0.0,1,1
8,00003eca-954b-4150-aee1-63fc62f395cf_7,HOMEPAGE,339790.0,MOBILE,LinkAja,0,NaN,1.0,0.0,0.0,1.0,0.0,4,4
9,00003eca-954b-4150-aee1-63fc62f395cf_7,PROMO_PAGE,339790.0,MOBILE,LinkAja,0,NaN,1.0,0.0,0.0,1.0,0.0,4,4


In [60]:
merged_df['total_promo'] = merged_df.groupby('new_session_id')['promo_amount_x'].transform('sum')

In [61]:
merged_df['total_promo'].describe()

count    2.113016e+07
mean     2.151051e+03
std      1.157101e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      3.537180e+05
Name: total_promo, dtype: float64

In [62]:
# check_describe = merged_df.drop_duplicates(subset='new_session_id')

In [63]:
# check_describe[check_describe['total_amount'] > 0]['total_amount'].describe()

In [64]:
def func(x):
    if x <= 0:
        return 0 
    elif x <= 2.037932e+05:
        return 1
    elif x <= 3.029890e+05:
        return 2
    elif x <= 5.147018e+05:
        return 3
    else:
        return 4


merged_df['total_amount'] = merged_df['total_amount'].apply(func)

In [65]:
# check_describe[check_describe['total_promo'] > 0]['total_promo'].describe()

In [66]:
def func(x):
    if x <= 0:
        return 0 
    elif x <= 3721.000000:
        return 1
    elif x <= 5404.000000:
        return 2
    elif x <= 8089.500000:
        return 3
    else:
        return 4


merged_df['total_promo'] = merged_df['total_promo'].apply(func)

In [67]:
merged_df = merged_df.drop(['event_name', 'traffic_source', 'payment_method', 'promo_amount_x'], axis=1)

In [68]:
merged_df = merged_df.drop_duplicates(subset='new_session_id')

In [69]:
time_df = time_df.drop(['first_join_date', 'created_at', 'target'], axis=1)

In [70]:
time_df.head(5)

,customer_id,new_session_id,diff
321,3.0,a857dcbe-5ef9-47a7-9fd6-478de1ffb635_5563287,NaT
409,8.0,1b8848eb-5ebc-4cbf-bf42-64c86b62c9d6_907932,NaT
447,9.0,6b23a139-8b85-4e68-a270-082ddf127cc4_3529286,NaT
448,11.0,5f20fb20-8e35-4e30-a9d9-fc6a07d7835c_3128683,NaT
487,15.0,cb1bb055-7431-4a77-b3e5-c896a9462aeb_6712949,NaT


In [71]:
time_df = time_df.merge(merged_df, on = 'new_session_id', how = 'left')

In [72]:
time_df['limit_ship'] = time_df['limit_ship'].astype(int)

In [73]:
time_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50704 entries, 0 to 50703
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   customer_id        50704 non-null  float64        
 1   new_session_id     50704 non-null  object         
 2   diff               0 non-null      timedelta64[ns]
 3   total_amount       50704 non-null  int64          
 4   shipment_fee       50704 non-null  int64          
 5   ADD_TO_CART_COUNT  50704 non-null  float64        
 6   SEARCH_COUNT       50704 non-null  float64        
 7   ADD_PROMO_COUNT    50704 non-null  float64        
 8   BOOKING_COUNT      50704 non-null  float64        
 9   ITEM_DETAIL_COUNT  50704 non-null  float64        
 10  limit_ship         50704 non-null  int32          
 11  PAGE_VIEW          50704 non-null  int64          
 12  total_promo        50704 non-null  int64          
dtypes: float64(6), int32(1), int64(4), object(1), 

In [74]:
time_df.head(10)

,customer_id,new_session_id,diff,total_amount,shipment_fee,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW,total_promo
0,3.0,a857dcbe-5ef9-47a7-9fd6-478de1ffb635_5563287,NaT,1,0,0.0,0.0,0.0,1.0,0.0,1,1,0
1,8.0,1b8848eb-5ebc-4cbf-bf42-64c86b62c9d6_907932,NaT,4,0,0.0,0.0,0.0,1.0,0.0,1,1,0
2,9.0,6b23a139-8b85-4e68-a270-082ddf127cc4_3529286,NaT,3,1,0.0,0.0,0.0,1.0,0.0,3,1,0
3,11.0,5f20fb20-8e35-4e30-a9d9-fc6a07d7835c_3128683,NaT,1,0,4.0,0.0,1.0,1.0,2.0,1,14,4
4,15.0,cb1bb055-7431-4a77-b3e5-c896a9462aeb_6712949,NaT,1,2,0.0,0.0,0.0,1.0,0.0,4,1,0
5,17.0,3816d1c4-29ad-4d7c-8bd1-23bfefdcb6a7_1847428,NaT,1,2,0.0,0.0,0.0,1.0,0.0,1,1,0
6,18.0,805ba729-58e8-4ae1-ba00-d8fb2427d5ef_4236063,NaT,1,2,0.0,0.0,0.0,1.0,0.0,3,1,0
7,20.0,3808ee5d-bca9-47d5-bddc-c481e09b796e_1845137,NaT,4,2,0.0,0.0,0.0,2.0,0.0,1,2,0
8,21.0,b13c7160-5ccf-46bb-9b57-be56946b3553_5855436,NaT,3,0,4.0,0.0,0.0,1.0,1.0,3,8,0
9,24.0,bfeb4bf5-fa47-4192-b405-5d9a40ec653e_6342284,NaT,2,3,0.0,0.0,0.0,1.0,0.0,3,1,0


In [75]:
time_df['total_session'] = time_df.groupby('customer_id')['new_session_id'].transform('count')

In [76]:
time_df.head(10)

,customer_id,new_session_id,diff,total_amount,shipment_fee,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW,total_promo,total_session
0,3.0,a857dcbe-5ef9-47a7-9fd6-478de1ffb635_5563287,NaT,1,0,0.0,0.0,0.0,1.0,0.0,1,1,0,1
1,8.0,1b8848eb-5ebc-4cbf-bf42-64c86b62c9d6_907932,NaT,4,0,0.0,0.0,0.0,1.0,0.0,1,1,0,1
2,9.0,6b23a139-8b85-4e68-a270-082ddf127cc4_3529286,NaT,3,1,0.0,0.0,0.0,1.0,0.0,3,1,0,1
3,11.0,5f20fb20-8e35-4e30-a9d9-fc6a07d7835c_3128683,NaT,1,0,4.0,0.0,1.0,1.0,2.0,1,14,4,1
4,15.0,cb1bb055-7431-4a77-b3e5-c896a9462aeb_6712949,NaT,1,2,0.0,0.0,0.0,1.0,0.0,4,1,0,1
5,17.0,3816d1c4-29ad-4d7c-8bd1-23bfefdcb6a7_1847428,NaT,1,2,0.0,0.0,0.0,1.0,0.0,1,1,0,1
6,18.0,805ba729-58e8-4ae1-ba00-d8fb2427d5ef_4236063,NaT,1,2,0.0,0.0,0.0,1.0,0.0,3,1,0,1
7,20.0,3808ee5d-bca9-47d5-bddc-c481e09b796e_1845137,NaT,4,2,0.0,0.0,0.0,2.0,0.0,1,2,0,1
8,21.0,b13c7160-5ccf-46bb-9b57-be56946b3553_5855436,NaT,3,0,4.0,0.0,0.0,1.0,1.0,3,8,0,1
9,24.0,bfeb4bf5-fa47-4192-b405-5d9a40ec653e_6342284,NaT,2,3,0.0,0.0,0.0,1.0,0.0,3,1,0,1


In [77]:
time_df['total_session'].describe()

count    50704.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: total_session, dtype: float64

In [78]:
# def func(x):
#     if x <= 1:
#         return 1
#     elif x <= 2:
#         return 2
#     elif x <= 4:
#         return 3
#     else:
#         return 4


# time_df['total_session'] = time_df['total_session'].apply(func)

In [79]:
time_df.to_csv('not_label_model.csv')

In [80]:
time_df['PAGE_VIEW'].describe()

count    50704.000000
mean         7.100584
std         26.003030
min          1.000000
25%          1.000000
50%          1.000000
75%          5.000000
max       1247.000000
Name: PAGE_VIEW, dtype: float64

In [81]:
def func(x):
    if x < 1:
        return 0
    elif x <= 1:
        return 1
    elif x <= 2:
        return 2
    else:
        return 3


time_df['PAGE_VIEW'] = time_df['PAGE_VIEW'].apply(func)

In [82]:
time_df['SEARCH_COUNT'].describe()

count    50704.000000
mean         0.403735
std          1.943658
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         96.000000
Name: SEARCH_COUNT, dtype: float64

In [83]:
def func(x):
    if x < 1:
        return 0
    elif x <= 1:
        return 1
    else:
        return 2


time_df['SEARCH_COUNT'] = time_df['SEARCH_COUNT'].apply(func)
time_df['ITEM_DETAIL_COUNT'] = time_df['ITEM_DETAIL_COUNT'].apply(func)
time_df['ADD_PROMO_COUNT'] = time_df['ADD_PROMO_COUNT'].apply(func)
time_df['ADD_TO_CART_COUNT'] = time_df['ADD_TO_CART_COUNT'].apply(func)

In [84]:
df = time_df.drop(['customer_id', 'new_session_id'], axis = 1)

In [85]:
# check_df에서 diff칼럼이 15일 이상인 행 추출
mask = df['diff'] > pd.Timedelta(days=30)
# target 열 생성
df['target'] = np.where(mask, 1, 0)

In [86]:
df.corr()

,diff,total_amount,shipment_fee,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW,total_promo,total_session,target
diff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
total_amount,NaN,1.000000,0.230647,0.015279,0.020040,0.035726,0.475789,0.032226,0.399083,0.216597,0.025162,NaN,NaN
shipment_fee,NaN,0.230647,1.000000,-0.485903,-0.265816,-0.258876,0.109323,-0.278596,0.215635,-0.352901,-0.247459,NaN,NaN
ADD_TO_CART_COUNT,NaN,0.015279,-0.485903,1.000000,0.512038,0.516320,0.023022,0.542417,-0.003338,0.703566,0.490494,NaN,NaN
SEARCH_COUNT,NaN,0.020040,-0.265816,0.512038,1.000000,0.366063,0.022752,0.592600,0.005034,0.478653,0.342208,NaN,NaN
ADD_PROMO_COUNT,NaN,0.035726,-0.258876,0.516320,0.366063,1.000000,0.058639,0.341908,0.004466,0.379844,0.929476,NaN,NaN
BOOKING_COUNT,NaN,0.475789,0.109323,0.023022,0.022752,0.058639,1.000000,0.030805,0.195398,0.343255,0.046490,NaN,NaN
ITEM_DETAIL_COUNT,NaN,0.032226,-0.278596,0.542417,0.592600,0.341908,0.030805,1.000000,0.013869,0.526251,0.320471,NaN,NaN
limit_ship,NaN,0.399083,0.215635,-0.003338,0.005034,0.004466,0.195398,0.013869,1.000000,0.039283,0.006668,NaN,NaN
PAGE_VIEW,NaN,0.216597,-0.352901,0.703566,0.478653,0.379844,0.343255,0.526251,0.039283,1.000000,0.363010,NaN,NaN


In [87]:
df.head(5)

,diff,total_amount,shipment_fee,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW,total_promo,total_session,target
0,NaT,1,0,0,0,0,1.0,0,1,1,0,1,0
1,NaT,4,0,0,0,0,1.0,0,1,1,0,1,0
2,NaT,3,1,0,0,0,1.0,0,3,1,0,1,0
3,NaT,1,0,2,0,1,1.0,2,1,3,4,1,0
4,NaT,1,2,0,0,0,1.0,0,4,1,0,1,0


In [88]:
# customer_id별로 total_session 칼럼에 번호 메기기
time_df['total_session'] = time_df.groupby('customer_id')['total_session'].rank(method='first')

In [89]:
time_df.tail(50)

,customer_id,new_session_id,diff,total_amount,shipment_fee,ADD_TO_CART_COUNT,SEARCH_COUNT,ADD_PROMO_COUNT,BOOKING_COUNT,ITEM_DETAIL_COUNT,limit_ship,PAGE_VIEW,total_promo,total_session
50654,99900.0,1eee1fb7-8ba1-4fea-a1cf-5516a3b668dd_1019850,NaT,2,2,0,0,0,1.0,0,4,1,0,1.0
50655,99901.0,d7229648-4650-4caf-b95b-68e377aa60b0_7115013,NaT,2,2,0,0,0,1.0,0,2,1,0,1.0
50656,99902.0,53c142a5-708c-4474-8514-312097c8aac2_2758096,NaT,3,3,0,0,0,1.0,0,3,1,0,1.0
50657,99905.0,f7ab5caa-f9ad-4d96-ae81-d56c9ea76abe_8192750,NaT,1,0,2,0,0,1.0,2,3,3,0,1.0
50658,99909.0,8db926ed-093d-4271-8178-95028ecb8961_4677784,NaT,0,0,0,0,0,0.0,0,0,1,0,1.0
50659,99910.0,8d7a585f-8224-4491-8326-9fadcd6fef5b_4669730,NaT,1,0,0,0,0,1.0,0,4,1,0,1.0
50660,99914.0,061ab3dd-081b-4f66-9eab-294e14166914_201013,NaT,2,3,0,0,0,1.0,0,2,1,0,1.0
50661,99915.0,c4b9b6c5-00a8-44c9-b035-c6a7c8819906_6504119,NaT,3,1,0,0,0,1.0,0,3,1,0,1.0
50662,99918.0,c301eab0-f765-4017-bd2d-fa272c6848f4_6446305,NaT,2,1,0,0,0,1.0,0,3,1,0,1.0
50663,99920.0,7bd16b3f-2527-443e-9eb3-2d994d4225c0_4085971,NaT,3,2,0,0,0,1.0,0,4,1,0,1.0


In [90]:
df = time_df.drop(['customer_id', 'new_session_id'], axis = 1)
# check_df에서 diff칼럼이 30일 이상인 행 추출
mask = df['diff'] > pd.Timedelta(days=30)
# target 열 생성
df['target'] = np.where(mask, 1, 0)

In [91]:
df['target'].value_counts()

target
0    50704
Name: count, dtype: int64

In [92]:
df.corr()['target']

diff                NaN
total_amount        NaN
shipment_fee        NaN
ADD_TO_CART_COUNT   NaN
SEARCH_COUNT        NaN
ADD_PROMO_COUNT     NaN
BOOKING_COUNT       NaN
ITEM_DETAIL_COUNT   NaN
limit_ship          NaN
PAGE_VIEW           NaN
total_promo         NaN
total_session       NaN
target              NaN
Name: target, dtype: float64

In [93]:
df.to_csv('droped_30days_target.csv')